<a href="https://colab.research.google.com/github/phukon/notebooks/blob/main/dspy_multihop_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dspy

In [29]:
import dspy
from dsp.utils import deduplicate

turbo = dspy.OpenAI(model='gpt-3.5-turbo-0125', api_key='xxx')
colbertv2_wiki_abstracts = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")
dspy.settings.configure(lm=turbo, rm=colbertv2_wiki_abstracts)

In [30]:
class GenerateQuery(dspy.Signature):
  """Generate search query from input question"""
  context = dspy.InputField(desc="may contain relevant facts")
  question = dspy.InputField()
  query = dspy.OutputField()

class GenerateAnswer(dspy.Signature):
  """Answer questions with short factoid answers"""
  context = dspy.InputField(desc="may contain relevant facts")
  question = dspy.InputField()
  answer = dspy.OutputField(des="often between 1 and 5 words")

In [34]:
class MultiHop(dspy.Module):
  def __init__(self, passages_per_hop=3, max_hops=3):
    super().__init__()

    self.generate_query = [dspy.ChainOfThought(GenerateQuery) for _ in range(max_hops)]
    self.retrieve = dspy.Retrieve(k=passages_per_hop)
    self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    self.max_hops = max_hops

  def forward(self, question):
    context = []
    for hop in range(self.max_hops):
      query_reponse = self.generate_query[hop](context=context, question=question)
      query = query_reponse.query
      passages = self.retrieve(query).passages
      context = deduplicate(context + passages)

    pred = self.generate_answer(context=context, question=question)
    return dspy.Prediction(context=context, answer=pred.answer)

In [37]:
question = "in which district is the last capital of the ahom kingdom located?"
multihop = MultiHop()
output = multihop(question=question)


print("Context: \n")
_ = [print(c) for c in output.context]

print("\nAnswer: ", output.answer)

Context: 

Rangpur (Ahom capital) | Rangpur (pron: ˈræŋpʊə or ˈræŋgpʊə), one of the capitals of the Ahom kingdom, was established by the Swargadeo Rudra Singha in 1707 . It is currently a part of Sibsagar town. Many historical monuments of the Ahom era, the Talatal Ghar and Rang Ghar, are here.
Bakata | Bakata was the capital of the Ahom kingdom (present-day Assam, Northeast India) established by the Ahom king Suhungmung. Since this capital was by the Dihing river, Suhungmung is also known as the "Dihingia Raja". The next king, Suklenmung, moved the capital to Garhgaon.
Garhgaon | Gargaon (Pron:/gɑ:ˈgɑ̃ʊ/) was the capital of the Ahom kingdom for many years. It was built by the Ahom king Suklenmung (Gargoyaan Rojaa) in 1540. It lies 13 km east of present-day Sivasagar town. The palace structures were made of wood and stones. In 1747 Pramatta Singha, son of Rudra Singha, constructed the brick wall of about 5 km in length surrounding the Gargaon Palace and the masonry gate leading to it.


In [39]:
turbo.inspect_history(6)





Generate search query from input question

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the query}. We ...

Query: ${query}

---

Context:
[1] «Rangpur (Ahom capital) | Rangpur (pron: ˈræŋpʊə or ˈræŋgpʊə), one of the capitals of the Ahom kingdom, was established by the Swargadeo Rudra Singha in 1707 . It is currently a part of Sibsagar town. Many historical monuments of the Ahom era, the Talatal Ghar and Rang Ghar, are here.»
[2] «Bakata | Bakata was the capital of the Ahom kingdom (present-day Assam, Northeast India) established by the Ahom king Suhungmung. Since this capital was by the Dihing river, Suhungmung is also known as the "Dihingia Raja". The next king, Suklenmung, moved the capital to Garhgaon.»
[3] «Garhgaon | Gargaon (Pron:/gɑ:ˈgɑ̃ʊ/) was the capital of the Ahom kingdom for many years. It was built by the Ahom king Suklenmung (Gargoyaan Rojaa) in 1540. It lies 1